# Embeddings

In [1]:
import sys  
sys.path.insert(0, 'tutorial')
from tutorial_setup import *

ALIAS: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/aliases.en.tsv.gz"
ALL: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/all.tsv.gz"
CLAIMS: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/claims.tsv.gz"
DESCRIPTION: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/descriptions.en.tsv.gz"
EXAMPLES_DIR: "/Users/pedroszekely/Documents/GitHub/kgtk/examples"
GE: "/Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding"
ISA: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/derived.isa.tsv.gz"
ITEM: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/claims.wikibase-item.tsv.gz"
KGTK_PATH: "/Users/pedroszekely/Documents/GitHub/kgtk"
LABEL: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/labels.en.tsv.gz"
OUT: "/Users/pedroszekely/Downloads/kgtk-tutorial/output"
P279: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/derived.P279.tsv.gz"
P279STAR: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/de

In [2]:
%cd {output_path}

/Users/pedroszekely/Downloads/kgtk-tutorial


In [20]:
!cp $text_embedding_path $TE

## Graph Embeddings

Normally, we would use `Q154ITEM`, but the partioning failed so we will compute it using kypher

Amandeep, Jan 14, 2021: Partition succeeded, change this?

In [4]:
os.environ["Q154GRAPH"] = os.environ["TEMP"] + "/Q154.edges.4.tsv.gz"

In [5]:
!zcat < "$Q154ITEM" | head

id	node1	label	node2	node2;wikidatatype
P10-P1855-Q7378-555592a4-0	P10	P1855	Q7378	wikibase-item
P1001-P1855-Q181574-7f428c9b-0	P1001	P1855	Q181574	wikibase-item
P1001-P1855-Q29868931-76b67d84-0	P1001	P1855	Q29868931	wikibase-item
P1001-P1855-Q3917299-6a038117-0	P1001	P1855	Q3917299	wikibase-item
P1001-P1855-Q7889769-df478289-0	P1001	P1855	Q7889769	wikibase-item
P1001-P1855-Q8901-15be5b36-0	P1001	P1855	Q8901	wikibase-item
P1004-P1855-Q209651-5b585f66-0	P1004	P1855	Q209651	wikibase-item
P1004-P1855-Q5471-0ca2dad9-0	P1004	P1855	Q5471	wikibase-item
P1005-P17-Q45-fa9c1f36-0	P1005	P17	Q45	wikibase-item
zcat: error writing to output: Broken pipe


In [6]:
!zcat < "$Q154GRAPH" | wc

  185238  764122 10113585


In [7]:
!$kypher -i "$Q154GRAPH" -i "$TEMP"/Q154.metadata.property.datatype.tsv.gz -i "$Q154LABEL" \
--match 'edges: (n1)-[l {label: property}]->(n2), datatype: (property)-[]->(dt:`wikibase-item`), label: (n1)-[]->(lab)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$GE"/geinput.tsv

We have over 60,000 lines:

In [8]:
!wc "$GE"/geinput.tsv

   59331  237324 2927769 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/geinput.tsv


Compute the graph embeddings using the default settings. Our output file `translation.txt` will be in word2vec format so we can usi it diectly in gensim

In [9]:
!$kgtk graph-embeddings --verbose -i "$GE"/geinput.tsv \
-o "$GE"/embeddings.txt \
--retain_temporary_data True \
--operator translation \
--workers 5 \
--log "$GE"/ge.log \
-T "$GE" \
-ot w2v \
-e 600

In Processing, Please go to /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/ge.log to check details
Opening the input file: /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/geinput.tsv
KgtkReader: File_path.suffix: .tsv
KgtkReader: reading file /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/geinput.tsv
header: id	node1	label	node2
node1 column found, this is a KGTK edge file
KgtkReader: Special columns: node1=1 label=2 node2=3 id=0
KgtkReader: Reading an edge file.
Opening the output file: /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/tmp_geinput.tsv
File_path.suffix: .tsv
KgtkWriter: writing file /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/tmp_geinput.tsv
header: id	node1	label	node2
Processing the input records.
Processed 59330 records.
Processed Finished.


Let's look at the output direcory

In [10]:
!ls -hl "$GE"

total 103280
-rw-r--r--   1 pedroszekely  staff    46M Jan 24 13:34 embeddings.txt
-rw-r--r--   1 pedroszekely  staff   949K Jan 24 13:34 ge.log
-rw-r--r--   1 pedroszekely  staff   2.8M Jan 24 13:29 geinput.tsv
drwxr-xr-x  10 pedroszekely  staff   320B Jan 24 13:33 output
-rw-r--r--   1 pedroszekely  staff   1.1M Jan 24 13:29 tmp_geinput.tsv


Let's peek at the file, we have 44K vectors of dimension 100

In [11]:
!head -2 "$GE"/embeddings.txt

37950 100
Q16963 -0.164185032 -0.385678679 -0.420757115 -0.225723416 -0.198370636 -0.639363647 -1.255813837 0.270225614 -0.272446662 0.066466041 -0.684449971 -0.646225393 0.655371726 0.400233626 -0.804983258 -0.407879442 -0.515173197 -1.094875097 -0.136672303 0.168045580 0.860949099 0.058952466 -0.393605471 0.034964170 -0.314580619 -0.375688851 0.132073671 -0.325051397 0.428311199 -0.724577427 -0.407612741 -0.259547085 0.170578405 0.244398162 0.578736067 -0.493021011 0.049207758 -0.011272416 0.574268937 -0.153083041 0.162458181 0.404403090 0.492393345 0.456173748 0.121200345 -0.969949603 0.057622679 0.437244385 -0.086911105 0.005650789 0.389287740 -0.682103992 -0.035925489 -0.142533764 0.495400161 -0.074574098 0.899632275 -0.513101518 0.876668990 -0.269911796 -0.116927855 -0.441026568 0.226159930 -0.396051615 0.553563178 0.396098971 -0.880450726 0.766135514 -0.105848886 -0.249817505 0.453038782 0.779053926 -0.202684402 -0.932523191 0.248839334 0.450294167 -0.427730024 0.355414480 0.755

Load the vectors in gensim

In [12]:
path = os.environ['GE'] + "/embeddings.txt"
ge_vectors = KeyedVectors.load_word2vec_format(path, binary=False)

In [13]:
# Q502268 is Johnnie Walker
ge_vectors['Q502268']

array([-0.13319838, -0.31249306,  0.5444786 ,  0.2620247 , -0.21586373,
        0.4774543 ,  0.35513186,  0.25343668,  0.62720376, -0.64378893,
       -0.1334848 ,  0.37392113, -0.05861791, -0.49614078,  0.00418998,
       -0.38867894, -0.11065244, -0.26280305,  0.03402166,  0.6086037 ,
        0.15529938, -0.47691786,  0.36543345, -1.0218014 ,  0.47811472,
       -0.46090993,  0.69470024,  1.2210124 ,  0.9414259 , -0.14485389,
       -0.58419716, -0.33650464, -0.00363764,  0.40011287, -0.6029851 ,
       -0.53706914, -0.38429427, -0.7802921 , -0.12297965, -0.33499527,
       -0.33098102,  0.60647637,  0.42458817,  0.26405805, -0.21734366,
        0.4993593 ,  0.2920877 , -0.5102409 , -0.07359102,  0.2263889 ,
       -0.31796598,  0.11023172, -0.24173269,  0.7149039 ,  0.51946294,
        0.79202014, -0.36261937, -0.925887  ,  1.0097575 ,  0.35304487,
        0.468682  , -0.1106649 ,  0.06289238,  0.30957982,  0.57368666,
       -0.3236156 , -0.02455906, -0.18257451, -0.01503056, -0.60

Find the most similar qnodes to `Q15874936`, the qnode for Michelob.

In [14]:
ge_vectors.most_similar(positive=['Q15874936'], topn=5)

[('P1562', 0.8587111234664917),
 ('Q7302072', 0.8569362163543701),
 ('P2174', 0.8532211184501648),
 ('Q2567026', 0.8531550168991089),
 ('Q5647008', 0.8489059209823608)]

This is hard to use because the reuslt are qnodes and we have no idea what they are. Let's define a function to fetch the labels and descriptions so that we can interpret the results more easily

`kgtk_most_similar` is a wrapper to gensim's `most_similar` function, and it is designed to output the results in KGTK format. The `kgtk_path` is required if we want to output the labels and descriptios as this path is where the `labels.en.tsv.gz` and `descriptions.en.tsv.gz` files care stored. You can optionally provide a `output_path` to tell it to sotre the results in a file; otherwise the results will be returned as a dataframe.

In [15]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    kg_path=None,
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """"""
    result = []
    if add_label_description and kg_path:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_label_graph"] = kg_path + "/labels.en.tsv.gz"
        os.environ["_description_graph"] = kg_path + "/descriptions.en.tsv.gz"
        os.environ["_temp_file"] = filename

        result = !$kypher_raw -i "$_label_graph" -i "$_description_graph" -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

Let's give it a try:

In [16]:
# Q15874936 is Michelob
kgtk_most_similar(ge_vectors, positive=['Q15874936'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,P1562,0.8587111234664917,similarity,'AllMovie title ID'@en,'identifier for a work on the website AllMovie...
1,P2174,0.8532211184501648,similarity,'Museum of Modern Art artist ID'@en,'identifier assigned to an artist by the Museu...
2,P4953,0.8437308073043823,similarity,'Library of Congress Genre/Form Terms ID'@en,'ID in the Library of Congress controlled voca...
3,Q2397992,0.8411049842834473,similarity,'malt liquor'@en,'beer style'@en
4,Q694536,0.8376587629318237,similarity,'American whiskey'@en,'Whiskey produced in the United States'@en
5,Q4912182,0.8343247771263123,similarity,'Billy Beer'@en,'beer produced in the United States'@en
6,Q5149389,0.8277002573013306,similarity,'Colt 45'@en,'malt liquor'@en


## Text embeddings

In [17]:
!zcat < $OUT/all.tsv.gz | head

id	node1	label	node2
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508
P10-P1659-P1651-c4068028-0	P10	P1659	P1651
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238
P10-P1659-P51-86aca4c5-0	P10	P1659	P51
P10-P1855-Q7378-555592a4-0	P10	P1855	Q7378
P10-P2302-Q21502404-d012aef4-0	P10	P2302	Q21502404
zcat: error writing to output: Broken pipe


The `kgtk text-embedding` command computes sentence vectors for each Qnode in the knowledge graph. The input to this command is a sorted KGTK edge file.

This is a two step process,

**Create a sentence for a Qnode using user specified properties**

In the command below, we have specified the following options,

- `--label-properties label` specifies that the property `label` has the label for the Qnode.
- `--isa-properties P31 P279 P452 P106` specified that `instance of` for the Qnode is defined by the propeties `P31 P279 P452 P106`
- `--description-properties description` specifies that the property `description` has the description for the Qnode.
- `--property-value P186 P17 P127 P176 P169` tells the command to use property-label and values from the properties `P186 P17 P127 P176 P169` to add additional context to sentence for the Qnode

Example sentence here

**Compute sentence vector using the sentence created in the previous step**

The command then computes a vector for the sentence using one of the models, specified as,
- `--model bert-large-nli-cls-token`

For more information on this command, please [click here](https://kgtk.readthedocs.io/en/latest/analysis/text_embedding/)


Duration --parallel 1
15300.45 real     14970.96 user       261.58 sys

The text embeddings are output in KGTK format and we need them in word2vec format (need to enhance the command to produce w2v format). For now, define a function to convert the KGTK embeddings to w2v format.

In [18]:
def convert_kgtk_to_w2v(input_path, output_path, text_embedding_label="text_embedding"):
    """
    Convert a KGTK file (node1/label/node2) that contains embeddings to the w2v format
    """
    vector_count = 0
    vector_length = 0
    
    # Read the file once to count the lines as we need to put them at the top of the w2v file
    with open(input_path, "r") as kgtk_file:
        next(kgtk_file)
        for line in kgtk_file:
            items = line.split("\t")
            qnode = items[0]
            label = items[1]
            if label == text_embedding_label:
                if vector_count == 0:
                    vector_length = len(items[2].split(","))
                vector_count += 1
        kgtk_file.close()

    with open(output_path, "w") as w2v_file:
        w2v_file.write("{} {}\n".format(vector_count, vector_length))
        with open(input_path, "r") as kgtk_file:
            next(kgtk_file)
            for line in kgtk_file:
                items = line.split("\t")
                qnode = items[0]
                label = items[1]
                if label == text_embedding_label:
                    vector = items[2].replace(",", " ")
                    w2v_file.write(qnode + " " + vector)
            kgtk_file.close()
        w2v_file.close()

In [21]:
convert_kgtk_to_w2v(os.environ['TE'] + "/text-embedding.tsv", os.environ['TE'] + "/embeddings.txt")

Let's look at the output file, the embeddings have 1024 dimensions

In [22]:
!head -10 "$TE"/embeddings.txt | tail -2

Q99970346 -0.35467714 0.11079551 -0.011766396 -0.64368856 0.7587074 -0.029240295 -0.34339845 -0.06344555 -1.6708547 0.388923 0.016877629 -0.016170679 0.17811266 1.0552806 -0.10560113 0.5062175 -0.37100965 -0.43509555 -0.7369594 0.9275887 0.6351612 -0.026170328 -0.6812031 -0.49427545 0.15076277 0.497177 -0.5669475 0.33832487 0.38121685 -0.34155178 -0.03627377 0.019129895 0.32135636 -1.3127131 0.2910208 -0.6110071 -0.21233878 -0.26547825 -0.48265418 0.19074659 -0.221765 -0.6583791 0.26793227 0.106484234 -0.51117957 -0.9209578 -0.53469723 -0.8773248 -0.50579745 0.28408417 -0.33325395 0.9733218 -0.20266499 1.2573 -0.67561316 -0.42509234 0.93198144 0.104132675 -0.72978777 0.61797714 0.5810334 0.11720219 -0.5360808 -1.2015952 0.31788793 -1.6091578 0.29825193 0.25895777 0.34890306 -0.64605564 -0.8923556 -0.6606609 0.27037627 0.13712278 0.047953844 0.9390667 -1.0347372 -1.0345485 0.6995126 0.8249064 0.35724065 0.27384388 -0.73517066 -0.35368446 -1.0148574 1.8248662 -0.07850242 1.4354324 0.5072

Load the text embeddings in gensim

In [23]:
te_path = os.environ['TE'] + "/embeddings.txt"
te_vectors = KeyedVectors.load_word2vec_format(te_path, binary=False)

### Compare the graph and text embeddings

Most similar nodes to Johnnie Walker using the **graph embeddings**

In [24]:
# Q502268 is Johnnie Walker
kgtk_most_similar(ge_vectors, positive=['Q502268'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q628737,0.8007535338401794,similarity,'Campbeltown Single Malts'@en,'single malt Scotch whiskies distilled in the ...
1,Q4865371,0.7964057326316833,similarity,'Bartlet for America'@en,'episode of The West Wing (S3 E9)'@en
2,P4553,0.7496455907821655,similarity,'RA Collections ID'@en,'identifier for an entry in the collections da...
3,Q209135,0.7390897274017334,similarity,'East Ayrshire'@en,'council area of Scotland'@en
4,P1430,0.734946608543396,similarity,'OpenPlaques subject ID'@en,'identifier for a person or other subject in t...
5,P5361,0.7194418907165527,similarity,'BNB person ID'@en,'identifier of a person in the British Nationa...
6,Q982891,0.7178372144699097,similarity,'The Macallan distillery'@en,'single malt whisky distillery'@en
7,Q955589,0.7153750658035278,similarity,'Coalville'@en,"'town in Leicestershire, England'@en"
8,Q187155,0.7090829014778137,similarity,'Tanqueray'@en,'trademark'@en


Most similar nodes to Johnnie Walker using the **text embeddings**

In [25]:
# Q502268 is Johnnie Walker
kgtk_most_similar(te_vectors, positive=['Q502268'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q280,0.9379171133041382,similarity,'Lagavulin Distillery'@en,"'Scotch whisky distillery in Lagavulin, Islay,..."
1,Q2490031,0.9346836805343628,similarity,'William Grant & Sons'@en,'Scottish company which distills Scotch whisky...
2,Q1543646,0.9012988805770874,similarity,'Rob Roy'@en,'cocktail based on Scotch whisky'@en
3,Q382947,0.8983699083328247,similarity,'Scotch whisky'@en,"'malt or grain whisky (or a blend of the two),..."
4,Q2168523,0.8907997012138367,similarity,'The Famous Grouse'@en,'brand of Scotch whisky'@en
5,Q1069502,0.8856704235076904,similarity,'Chivas Regal'@en,'Blended Scotch Whisky produced by Chivas Brot...
6,Q6744642,0.8838940858840942,similarity,'malt whisky'@en,"'Distilled spirit from Scotland (a/k/a \\\\""Sc..."
7,Q4821838,0.8762272596359253,similarity,'Aultmore distillery'@en,"'whisky distillery in Moray, Scotland, UK'@en"
8,Q1754978,0.8664095401763916,similarity,'Rusty Nail'@en,'cocktail mixing Drambuie and Scotch whisky'@en


The graph embeddings produce poor results as the top matches are not related to whiskey. The text embeddings look much better.

Most similar nodes to Michelob using the **graph embeddings**

In [26]:
# Q15874936 is Michelob
kgtk_most_similar(ge_vectors, positive=['Q15874936'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,P1562,0.8587111234664917,similarity,'AllMovie title ID'@en,'identifier for a work on the website AllMovie...
1,P2174,0.8532211184501648,similarity,'Museum of Modern Art artist ID'@en,'identifier assigned to an artist by the Museu...
2,P4953,0.8437308073043823,similarity,'Library of Congress Genre/Form Terms ID'@en,'ID in the Library of Congress controlled voca...
3,Q2397992,0.8411049842834473,similarity,'malt liquor'@en,'beer style'@en
4,Q694536,0.8376587629318237,similarity,'American whiskey'@en,'Whiskey produced in the United States'@en
5,Q4912182,0.8343247771263123,similarity,'Billy Beer'@en,'beer produced in the United States'@en
6,Q5149389,0.8277002573013306,similarity,'Colt 45'@en,'malt liquor'@en


Most similar nodes to Michelob using the **text embeddings**

In [27]:
# Q15874936 is Michelob
kgtk_most_similar(te_vectors, positive=['Q15874936'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q2011473,0.9664472341537476,similarity,'Fantôme'@en,'brand of beer'@en
1,Q3315575,0.9586231708526611,similarity,'Bersalis'@en,'beer brand'@en
2,Q3518554,0.9563601016998291,similarity,'Floris'@en,'beer brand'@en
3,Q15076069,0.9531255960464478,similarity,'Marckloff'@en,'beer brand'@en
4,Q1277388,0.951164722442627,similarity,'Pripps Blå'@en,'beer brand'@en
5,Q1917255,0.9475076794624329,similarity,'St-Idesbald'@en,'beer'@en
6,Q263980,0.9443504810333252,similarity,'Soproni'@en,'beer mark'@en


The graph embeddings contain some bad results, but the top matches are better as they include beers that are more closely related to Michelob. The text embeddings are reasonable as they include only beers.

Most similar nodes to vodka using the **graph embeddings**

In [28]:
# Q374 is vodka
kgtk_most_similar(ge_vectors, positive=['Q374'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q7468032,0.8597341775894165,similarity,'Vodka'@en,'Detective Conan character'@en
1,Q20577688,0.85808265209198,similarity,'.vodka'@en,'top-level Internet domain'@en
2,Q21189725,0.8241639733314514,similarity,'Red Eye Louie\\\\'s Vodquila'@en,'blend of vodka and tequila'@en
3,Q8692888,0.8167219161987305,similarity,'Category:Russian vodkas'@en,'Wikimedia category'@en
4,Q7151801,0.8038827180862427,similarity,'Category:Vodkas'@en,'Wikimedia category'@en
5,Q11328065,0.7957900762557983,similarity,'Balalaika'@en,"'Japanese short drink, cocktail'@en"
6,Q2206588,0.7952877283096313,similarity,'Caipiroska'@en,'cocktail prepared with vodka'@en
7,Q5134411,0.7506065368652344,similarity,'Clique'@en,'Latvian vodka brand'@en


Most similar nodes to vodka using the **text embeddings**

In [29]:
# Q374 is vodka
kgtk_most_similar(te_vectors, positive=['Q374'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q4869283,0.9598516821861267,similarity,'Batini'@en,'vodka-based cocktail'@en
1,Q3562046,0.9595369696617126,similarity,'Vodka Stinger'@en,'type of cocktail'@en
2,Q2206588,0.9436805248260498,similarity,'Caipiroska'@en,'cocktail prepared with vodka'@en
3,Q22236238,0.9384632110595703,similarity,'Mariette'@en,"'vodka, alcohol'@en"
4,Q7939317,0.9203516244888306,similarity,'Vodka Cruiser'@en,'brand of vodka-based alcoholic drink'@en
5,Q11802565,0.9155371785163879,similarity,'Pan Tadeusz'@en,'brand of vodka'@en
6,Q268057,0.9129105806350708,similarity,'cosmopolitan'@en,'cocktail made with vodka'@en
7,Q4782617,0.9107506275177002,similarity,'Aqua Velva'@en,'vodka and gin based cocktail'@en


The graph embeddings are noisy as the top matches include nodes not related to vodka, the text embeddings look much better.

Let's look at countries now as the differences between the two types of embeddings are more striking.
The graph embeddings retrieve nodes that are related to Ireland:

In [30]:
# Q27 Ireland
kgtk_most_similar(ge_vectors, positive=['Q27'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q162475,0.8572713732719421,similarity,'County Cork'@en,'county in Ireland'@en
1,Q186220,0.8410564661026001,similarity,'County Longford'@en,'county in Ireland'@en
2,Q164421,0.838392972946167,similarity,'Connacht'@en,'province in Ireland'@en
3,Q178283,0.837478756904602,similarity,'County Limerick'@en,'county in Ireland'@en
4,Q131438,0.8331077098846436,similarity,'Munster'@en,'province in Ireland'@en
5,Q184469,0.8307558298110962,similarity,'County Kerry'@en,'county in Ireland'@en
6,Q184594,0.8298261761665344,similarity,'County Waterford'@en,'county in Ireland'@en
7,Q178626,0.8277775049209595,similarity,'County Mayo'@en,'county in Ireland'@en
8,Q184760,0.8252919316291809,similarity,'County Monaghan'@en,'county in Ireland'@en
9,Q6334335,0.8023019433021545,similarity,'Category:Deaths in Ireland'@en,'Wikimedia category'@en


THe text embeddings retrieve other countries:

In [31]:
# Q27 Ireland
kgtk_most_similar(te_vectors, positive=['Q27'], kg_path=os.environ['OUT'] + "/parts", topn=10)

,node1,node2,label,node1;label,node1;description
0,Q191,0.7966251969337463,similarity,'Estonia'@en,'sovereign state in northeastern Europe'@en
1,Q37,0.7891267538070679,similarity,'Lithuania'@en,'sovereign state in northeastern Europe'@en
2,Q20,0.7881592512130737,similarity,'Norway'@en,'sovereign state in northern Europe'@en
3,Q34,0.7823097109794617,similarity,'Sweden'@en,'sovereign state in northern Europe'@en
4,Q35,0.7809572815895081,similarity,'Denmark'@en,'sovereign state in northern Europe that is pa...
5,Q33,0.7614077925682068,similarity,'Finland'@en,'sovereign state in northern Europe'@en
6,Q1526538,0.7550898194313049,similarity,'Reykjavík North'@en,'one of the six constituencies (kjördæmi) of I...
7,Q16965019,0.7516392469406128,similarity,'North borough of Brescia'@en,'one of 5 boroughs of Brescia'@en
8,Q189,0.7509456276893616,similarity,'Iceland'@en,"'sovereign state in Northern Europe, situated ..."
9,Q22,0.7428288459777832,similarity,'Scotland'@en,"'country in Northwest Europe, part of the Unit..."


### Using the embeddings in queries to the KG

In [32]:
# Q281 whiskey
# Q282 wine
# Q3246609 mixed drink
# Q374 vodka
# Q332378 is absolut

Get the most similar nodes to **absolut**, the swedish vodka using the text embeddings and put it in a file

In [33]:
# Q332378 is absolut
kgtk_most_similar(te_vectors, positive=['Q332378'], kg_path=os.environ['OUT'] + "/parts", topn=1000, output_path=os.environ['TE'] + "/Q332378.sim.tsv")

In [34]:
result = !head "$TE"/Q332378.sim.tsv
kgtk_to_dataframe(result)

,node1,node2,label,node1;label,node1;description
0,Q7312560,0.9494207501411438,similarity,'Renat'@en,'Swedish vodka'@en
1,Q406157,0.9068877696990967,similarity,'bäsk'@en,'Swedish style spiced liquor'@en
2,Q1034035,0.8990318775177002,similarity,'Finlandia Vodka'@en,'Finnish brand of vodka'@en
3,Q374,0.8908253908157349,similarity,'vodka'@en,'distilled alcoholic beverage'@en
4,Q2553569,0.8900324106216431,similarity,'vodka martini'@en,'cocktail made with vodka and vermouth'@en
5,Q2206588,0.8866581916809082,similarity,'Caipiroska'@en,'cocktail prepared with vodka'@en
6,Q268057,0.8860777616500854,similarity,'cosmopolitan'@en,'cocktail made with vodka'@en
7,Q4021706,0.8785414695739746,similarity,'Xan'@en,'Vodka from Goygol'@en
8,Q4869283,0.8784171938896179,similarity,'Batini'@en,'vodka-based cocktail'@en


Suppose I have absolut vodka and I want to make a cocktail. I can use the KG graph of the most similar nodes to absolut, and search the KG for mixed drinks (`Q3246609`) that appear in the list of most similar nodes to absolut.

Here are some drinks we can make with absolut vodka. The query starts with our similarity file (`Q332378.sim.tsv`) in clause `sim` and filters it to select the qnodes that are instances of mixed drink (`Q3246609`) using clauses `isa` and `star`. Then the first `claims` clause selects those that have vodka as an ingredient (`Q374`) and the second `claims` clause retrieves the other ingredients.

In [37]:
result = !$kypher -i "$ISA" -i "$P279STAR" -i "$TE"/Q332378.sim.tsv -i "$Q154CLAIMS" -i "$Q154LABEL" \
--match 'sim: (n1)-[]->(similarity), isa: (n1)-[]->(isa), star: (isa)-[]->(class), \
  claims: (n1)-[:P186]->(:Q374), claims: (n1)-[:P186]->(ingredient), label: (ingredient)-[]->(i_label)' \
--return 'distinct n1 as node1, similarity as node2, n1.label, n1.description, \
  ingredient as ingredient, i_label as `ingredient label`' \
--order-by 'cast(similarity, float) desc' \
--where 'class = "Q3246609"' \
--limit 20 

kgtk_to_dataframe(result)

,node1,node2,node1;label,node1;description,ingredient,ingredient label
0,Q2553569,0.8900324106216431,'vodka martini'@en,'cocktail made with vodka and vermouth'@en,Q1105343,'cocktail glass'@en
1,Q2553569,0.8900324106216431,'vodka martini'@en,'cocktail made with vodka and vermouth'@en,Q1621080,'olive'@en
2,Q2553569,0.8900324106216431,'vodka martini'@en,'cocktail made with vodka and vermouth'@en,Q26877166,'lemon twist'@en
3,Q2553569,0.8900324106216431,'vodka martini'@en,'cocktail made with vodka and vermouth'@en,Q26877423,'dry vermouth'@en
4,Q2553569,0.8900324106216431,'vodka martini'@en,'cocktail made with vodka and vermouth'@en,Q374,'vodka'@en
5,Q2206588,0.8866581916809082,'Caipiroska'@en,'cocktail prepared with vodka'@en,Q374,'vodka'@en
6,Q1966883,0.8709858655929565,'Yorsh'@en,'Russian drink of beer and vodka'@en,Q374,'vodka'@en
7,Q1966883,0.8709858655929565,'Yorsh'@en,'Russian drink of beer and vodka'@en,Q44,'beer'@en
8,Q1723060,0.8683922290802002,'Kamikaze'@en,"'cocktail of vodka, triple sec and lime juice'@en",Q1105343,'cocktail glass'@en
9,Q1723060,0.8683922290802002,'Kamikaze'@en,"'cocktail of vodka, triple sec and lime juice'@en",Q3539556,'triple sec'@en


The results are good, lots of choices of cocktails. Note that the embeddings are able to generalize from a specific vodka to vodka in general. The example also illustrates that KGTK can use the results of queries to gensim within queries to the KG.

**This cell sometimes does not produce results. Seems to be randomly working?**

When we try the query using the graph embeddings, and do not explictly filter the ingredients to include vodka:

In [38]:
# Q332378 is absolut
kgtk_most_similar(ge_vectors, positive=['Q332378'], kg_path=os.environ['OUT'] + "/parts", topn=2000, output_path=os.environ['GE'] + "/Q332378.sim.tsv")

In [40]:
result = !$kypher -i "$ISA" -i "$P279STAR" -i "$GE"/Q332378.sim.tsv \
--match 'sim: (n1)-[]->(similarity), isa: (n1)-[]->(isa), star: (isa)-[]->(class)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, n1.label, n1.description' \
--order-by 'cast(similarity, float) desc' \
--where 'class = "Q3246609"' \
--limit 10 

kgtk_to_dataframe(result)

,node1,node2,label,node1;label,node1;description
0,Q7633290,0.48373061418533325,similarity,'sudan'@en,'traditional Korean punch'@en
1,Q7084686,0.4789865016937256,similarity,'Old Pal'@en,"'cocktail of Canadian Rye Whiskey, Vermouth an..."
2,Q7271694,0.4633059501647949,similarity,'Quick Fuck'@en,'layered shooter (drink)'@en
3,Q2406926,0.44280266761779785,similarity,'Jägerbomb'@en,'A bomb shot mixed drink'@en
4,Q13527031,0.4344600439071655,similarity,'shandy'@en,'family of drinks made of beer mixed with a so...
5,Q33076550,0.41808271408081055,similarity,'Canadian Fashion'@en,'whiskey cocktail'@en


The results are poor as for the most part, the retrieved cocktails do not have vodka. Let's try the query with vodka instead of absolut vodka.

**This cell sometimes does not produce results. Seems to be randomly working, same as above?**

Now let get the qnodes that are similar to vodka (`Q374`) using the graph embeddings:

In [41]:
# Q374 vodka
kgtk_most_similar(ge_vectors, positive=['Q374'], kg_path=os.environ['OUT'] + "/parts", topn=1000, output_path=os.environ['GE'] + "/Q374.sim.tsv")

In [46]:
result = !$kypher_raw -i isa -i p279star -i "$GE"/Q374.sim.tsv \
--match 'sim: (n1)-[]->(similarity), isa: (n1)-[]->(isa), star: (isa)-[]->(class)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, n1.label, n1.description' \
--order-by 'cast(similarity, float) desc' \
--where 'class = "Q3246609"' \
--limit 10 

kgtk_to_dataframe(result)

,node1,node2,label,node1;label,node1;description
0,Q11328065,0.7957900762557983,similarity,'Balalaika'@en,"'Japanese short drink, cocktail'@en"
1,Q2206588,0.7952877283096313,similarity,'Caipiroska'@en,'cocktail prepared with vodka'@en
2,Q3562046,0.6846504211425781,similarity,'Vodka Stinger'@en,'type of cocktail'@en
3,Q3900577,0.6435666084289551,similarity,'Pertini'@en,'cocktail drink with honey'@en
4,Q4835871,0.6270689964294434,similarity,'BLT cocktail'@en,"'vodka cocktail with bacon, lettuce and tomato..."
5,Q26879480,0.5979952812194824,similarity,'Godmother'@en,'cocktail'@en
6,Q455914,0.5960879325866699,similarity,'Vodka Red Bull'@en,'alcoholic beverage'@en
7,Q5103598,0.5952239036560059,similarity,'Chocolate Cake'@en,'cocktail'@en
8,Q5459745,0.5844936966896057,similarity,'flirtini'@en,"'cocktail containing vodka, champagne and pine..."
9,Q7367818,0.5656362771987915,similarity,'Rose Kennedy'@en,"'cocktail with vodka, club soda and a splash o..."


The results are good. Somehow, the graph embeddings are able to rerieve the cocktails that have vodka, but cannot generalize from absolut vodka to vodka.

## Produce files to load in the Google Embedding Projector
The Goodle embedding projector (https://projector.tensorflow.org) is a tool for visualizing embeddings. To use it we need two files:

- a TSV file with the vectors
- a TSV file with the metadata, in the same order as the vectors

We don't want to load all the vectors in the projectors because it is too many to visualize. We will load only the following types as it will be interesting to see whether they cluster properly.

In [47]:
focus_types = {
    "Q3246609": "mixed drink",
    "Q44": "beer",
    "Q282": "wine",
    "Q281": "whiskey",
    "Q374": "vodka",
    "Q6256": "country",
}

To do the filteriing, we construct a dictionary that maps every q-node in the KG to the set of all its superclasses. We will use this dictionary later to tag each q-node with one of the focus types. For every q-node we will test if the focus type is in the set of all super-classes.

In [48]:
classes_result = !$kypher_raw -i "$ISA" -i "$Q154CLAIMS" -i "$TEMP"/Q154.descendant.tsv -i "$P279STAR" \
--match 'isa: (n1)-[]->(c), P279: (c)-[]->(class), claims: ()-[]->(class), descendant: (n1)-[]->()' \
--return 'distinct n1 as qnode, class as class' 

class_dict = {}
for r in classes_result[1:]:
    row = r.split("\t")
    qnode = row[0]
    isa = row[1]
    entry = class_dict.get(qnode)
    if entry is None:
        class_dict[qnode] = set()
        entry = class_dict[qnode]
    entry.add(isa)

Let's look at the class_dict for Johnnie Walker (`Q502268`). We see that Johnnie Walker has many super classes.

In [49]:
class_dict['Q502268']

{'Q102205',
 'Q107715',
 'Q11024',
 'Q11028',
 'Q111352',
 'Q11435',
 'Q1150070',
 'Q1166770',
 'Q11795009',
 'Q1190554',
 'Q1194058',
 'Q12767945',
 'Q131257',
 'Q13878858',
 'Q1400881',
 'Q1422299',
 'Q154',
 'Q15401930',
 'Q1554231',
 'Q15619164',
 'Q1632297',
 'Q16686448',
 'Q16722960',
 'Q167270',
 'Q1681365',
 'Q16887380',
 'Q16889133',
 'Q1704572',
 'Q174984',
 'Q1786828',
 'Q17988854',
 'Q187931',
 'Q1914636',
 'Q20817253',
 'Q20937557',
 'Q2095',
 'Q2150504',
 'Q22269697',
 'Q22272508',
 'Q22294683',
 'Q223557',
 'Q23009552',
 'Q23009675',
 'Q2424752',
 'Q246672',
 'Q25481995',
 'Q26907166',
 'Q27166344',
 'Q281',
 'Q28728771',
 'Q28732711',
 'Q28813620',
 'Q28877',
 'Q2944660',
 'Q29651519',
 'Q2990593',
 'Q2996394',
 'Q309314',
 'Q31464082',
 'Q3249551',
 'Q337060',
 'Q35120',
 'Q35758',
 'Q3695082',
 'Q382947',
 'Q386724',
 'Q40050',
 'Q4026292',
 'Q427581',
 'Q42848',
 'Q43460564',
 'Q4373292',
 'Q4406616',
 'Q4437984',
 'Q46737',
 'Q478798',
 'Q483247',
 'Q488383',
 'Q529

In [50]:
def focus_type(qnode):
    """
    Retrieve the focus type for any qnode, and return "other" for nodes that are not instances of our focus types.
    """
    for t in focus_types.keys():
        classes = class_dict.get(qnode)
        if classes and t in classes:
            return focus_types[t]
        if qnode in country_qnodes:
            return "country"
    return "other"

Construct `country_qnodes`, the set of all country qnodes

In [51]:
country_result = !$kypher_raw -i isa -i p279star -i "$Q154CLAIMS" \
--match 'claims: (country)-[]->(), isa: (country)-[:isa]->(c), P279: (c)-[]->(:Q6256)' \
--return 'distinct country as country' 

country_qnodes = set()
for r in country_result[1:]:
    country_qnodes.add(r)

Construct `alcoholic_qnodes`, the set of all alcoholic beverage qnodes.

In [52]:
alcoholic_qnodes = set()
for line in open(os.environ["TEMP"] + "/Q154.descendant.tsv", "r"):
    alcoholic_qnodes.add(line.split("\t")[0])

The `build_embedding_projector_vectors` builds the vectors file, a TSV file with one line for each vector. We do this by scanning through the full embeddings file and selecting qnodes that are in our set of `alcoholic_qnodes` or `coutnry_qnodes`. We also write a file of all the qnodes that select. We will use this file later to construct the metadat file. We have to be careful to list the qnodes in the metadata file in the same order as they appear in the vectors file.

In [53]:
def build_embedding_projector_vectors(embeddings_path):
    input_path = embeddings_path + "/embeddings.txt"
    vectors_path = embeddings_path + "/projector.vectors.tsv"
    qnodes_path = embeddings_path + "/projector.qnodes.tsv"

    input_file = open(input_path, "r")
    vectors_file = open(vectors_path, "w")
    qnodes_file = open(qnodes_path, "w")

    qnodes_file.write("node1\n")

    with open(input_path, "r") as w2v_file:
        next(w2v_file)
        for line in w2v_file:
            items = line.split(" ")
            qnode = items[0]
            if qnode in alcoholic_qnodes or qnode in country_qnodes:
                vectors_file.write("\t".join(items[1:]))
                qnodes_file.write("{}\n".format(qnode))

    input_file.close()
    vectors_file.close()
    qnodes_file.close()

In [55]:
build_embedding_projector_vectors(os.environ["GE"])

Let's take a peek at our qnodes file, which we use in the next step.

In [56]:
!head "$GE"/projector.qnodes.tsv

node1
Q2835719
Q11797012
Q4010098
Q220275
Q2945652
Q3006041
Q3909881
Q1398382
Q2835711


The `build_embedding_projector_metadata` uses a kypher query to retreive the labels of the qnodes (in a later version we will also include the descriptions; for now we don't because the query filters out qnodes that don't have descriptions, and unfortunaely, many alcoholic beverages are missing English descriptions).

The idea is:
- Retrieve the labels for all the qnodes using the kypher query. The query returns the results in arbitrary order.
- Build a dictionary that maps each node to the metadata that we want.
- Scan the qnodes file and for each qnode, write a metadata line in the metadata file (`projector.metadata.tsv`)

Our metadata file has three columns (you can have as many as you want):
- tag: includes the label and the focus type as it is often difficult to tell from the tag what type of beverage it is
- qnode
- focus type

In [57]:
def build_embedding_projector_metadata(embeddings_path):
    kg_path = os.environ["OUT"] + "/parts"
    os.environ["_label_graph"] = kg_path + "/labels.en.tsv.gz"
    os.environ["_description_graph"] = kg_path + "/descriptions.en.tsv.gz"
    os.environ["_qnodes"] = embeddings_path + "/projector.qnodes.tsv"

    #result = !$kypher_raw -i "$_label_graph" -i "$_description_graph" -i "$_qnodes" \
    #--match 'qnodes: (n1)-[]->(), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
    #--return 'distinct n1 as node1, lab as `node1;label`, des as `node1;description`' 
    
    result = !$kypher_raw -i "$_label_graph" -i "$_description_graph" -i "$_qnodes" \
    --match 'qnodes: (n1)-[]->(), label: (n1)-[]->(lab)' \
    --return 'distinct n1 as node1, lab as `node1;label`'
    
    metadata_path = embeddings_path + "/projector.metadata.tsv"
    metadata_file = open(metadata_path, "w")
    metadata_file.write("tag\tqnode\ttype\n")

    qnode_dict = {}
    for line in result[1:]:
        items = line.split("\t")
        qnode = items[0]
        # qnode_dict[qnode] = "{} ({})".format(items[1], items[2])
        qnode_dict[qnode] = "{}".format(items[1])

    with open(os.environ["_qnodes"]) as qnodes_file:
        next(qnodes_file)
        for line in qnodes_file:
            qnode = line[:-1]
            ftype = focus_type(qnode)
            tag = qnode_dict.get(qnode)
            if tag is None:
                tag = qnode
            tag = "{} ({})".format(qnode_dict.get(qnode), ftype)
            metadata_file.write("{}\t{}\t{}\n".format(tag, qnode, ftype))

    metadata_file.close()
    qnodes_file.close()       

In [58]:
build_embedding_projector_metadata(os.environ["GE"])

Check that the file sizes are correct, the metadata file has one more line as it as headers.

In [59]:
!wc "$GE"/projector.metadata.tsv "$GE"/projector.vectors.tsv

    2172   11418   92820 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/projector.metadata.tsv
    2171  217100 2720129 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding/projector.vectors.tsv
    4343  228518 2812949 total


In [60]:
!head -1 "$GE"/projector.vectors.tsv

-0.075612903	-0.050318368	0.298300803	0.421432823	-0.248541415	0.056908730	-0.680637777	-0.073326267	-0.377136022	-0.134014219	-0.197469816	0.202630788	-0.221844554	-0.245265618	0.065674670	-0.152094126	-0.199116051	0.087020665	0.324004859	0.213313386	-0.038387485	-0.210035563	0.164019927	-0.103406079	-0.427563787	0.039796054	-0.031909943	-0.089524582	0.020940976	0.067550369	-0.247933343	0.429589391	0.332343102	0.304964006	0.132957101	-0.267977208	0.027822251	0.224115863	0.144821435	0.192092970	-0.252484620	-0.062621295	0.487187892	-0.134187669	-0.354380071	-0.010176005	-0.099684849	-0.069353126	-0.344365478	-0.135896817	0.216197208	0.475384742	0.079261489	0.195580140	-0.030683421	0.130388871	0.029154558	0.147579566	0.127199963	0.265521586	0.290794969	-0.006078408	0.314150780	-0.056377053	0.767467082	-0.341108173	-0.338996470	0.261381328	-0.195630014	-0.060107719	-0.220251441	-0.136010066	-0.238110855	-0.007777404	0.293746799	-0.038727939	0.024510127	0.249717876	-0.326814592	0.21904832

Now build the projector files for the text embeddings, and check that the sizes are ok

In [61]:
build_embedding_projector_vectors(os.environ["TE"])

In [62]:
build_embedding_projector_metadata(os.environ["TE"])

In [63]:
!wc "$TE"/projector.metadata.tsv "$TE"/projector.vectors.tsv "$TE"/projector.qnodes.tsv

    2730   14366  116939 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/text-embedding/projector.metadata.tsv
    2729 2794496 31118184 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/text-embedding/projector.vectors.tsv
    2730    2730   24810 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/text-embedding/projector.qnodes.tsv
    8189 2811592 31259933 total


### Google embedding projector
- open https://projector.tensorflow.org
- Load your files using the load button
- configure the visualization

Here we searched on the right for absolut vodka, and we see the closest vecotrs as well as the cluster where it belongs:
![Google embedding projector](assets/embedding-projector.png "Google embedding projector")

### UMAP visualization of the graph embeddings


Very few vodkas, hard to see them in the visualization.


![UMAP visualization](assets/graph-embedding-umap-13.png "UMAP visualization of graph embeddings")

### UMAP visualization of the text embeddings
Very few vodkas, har to see them in the visualization.


![UMAP visualization](assets/text-embedding-umap-17.png "UMAP visualization of text embeddings")

### Text search using the text embeddings

In [64]:
from sentence_transformers import SentenceTransformer


class ComputeEmbeddings:
    def __init__(self, model_name=None):
        if not model_name:
            self.model_name = 'bert-large-nli-cls-token'
        else:
            self.model_name = model_name

        self.model = SentenceTransformer(self.model_name)

    def get_vectors(self, sentence):
        """
            main function to get the vector representations of the descriptions
        """
        if isinstance(sentence, bytes):
            sentence = sentence.decode("utf-8")
        return self.model.encode([sentence], show_progress_bar=False)

In [65]:
em = ComputeEmbeddings()

In [66]:
v = em.get_vectors("beer company")[0]

In [67]:
te_vectors.similar_by_vector(v)

[('Q6119795', 0.9004269242286682),
 ('Q22333354', 0.88853520154953),
 ('Q28771839', 0.878260612487793),
 ('Q878975', 0.8738116025924683),
 ('Q4880037', 0.8519435524940491),
 ('Q1637028', 0.8471935987472534),
 ('Q28530481', 0.8351479768753052),
 ('Q696787', 0.8316406607627869),
 ('Q20571254', 0.8302997350692749),
 ('Q460206', 0.829006552696228)]

In [69]:
!wd u Q6119795

id Q6119795
Label Jacobsen
Description beer brand
instance of (P31): brewery (Q131734)
